In [15]:
import numpy as np
import matplotlib.pyplot as plt
import random 

random.seed(42)

In [24]:



class Perceptron:
    def __init__(self, max_iters=100):
       self.max_iters = max_iters
       
    def fit(self, X, y):
        X, y = np.asarray(X), np.asarray(y)
        iters = 0
        
        #Coluna do bias
        X = np.concatenate(
            (X, np.asarray([[1] * X.shape[0]]).T),
            axis=1
        )
        
        #Weights aleatorios
        w = np.random.random(X.shape[1])
        
        for _ in range(self.max_iters):
            y_pred_all = []
            for idx in range(X.shape[0]):
                x_sample, y_sample = X[idx], y[idx]
                y_pred = int(np.sum(w * x_sample) >= 0.5)
                if y_pred == y_sample:
                    pass
                elif y_pred == 0 and y_sample == 1:
                    w = w + x_sample
                elif y_pred == 1 and y_sample == 0:
                    w = w - x_sample
                
                y_pred_all.append(y_pred)
                
            iters += 1
            if np.equal(np.array(y_pred_all), y).all():
                break
            
        self.iters, self.w = iters, w
                
    def predict(self, X):
        X = np.asarray(X)
        X = np.concatenate((X, np.asarray([[1] * X.shape[0]]).T), axis=1)
        return (X @ self.w >= 0.5).astype(int)
    
    def visualize_training(self, X, y):
        """
        Visualiza a evolução da reta de decisão durante o treinamento
        """
        if not hasattr(self, 'history') or len(self.history) == 0:
            print("Treine o modelo primeiro usando o método fit()")
            return
        
        # Preparar dados para visualização
        X_orig = X.copy()
        X = np.asarray(X)
        y = np.asarray(y)
        
        # Configurar o gráfico
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Plotar os pontos de dados
        for label in np.unique(y):
            mask = y == label
            ax.scatter(X[mask, 0], X[mask, 1], 
                      label=f'Classe {label}', 
                      s=100, 
                      alpha=0.7)
        
        # Função para desenhar a linha de decisão
        def draw_line(w, ax, min_x, max_x):
            # w0*x + w1*y + w2*bias = 0.5
            # y = (0.5 - w0*x - w2) / w1
            if w[1] == 0:  # Evitar divisão por zero
                return
            
            x = np.linspace(min_x, max_x, 100)
            y = (0.5 - w[0] * x - w[2]) / w[1]
            line, = ax.plot(x, y, 'r-', lw=2)
            return line
        
        # Definir limites do gráfico
        min_x, max_x = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
        min_y, max_y = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
        
        ax.set_xlim(min_x, max_x)
        ax.set_ylim(min_y, max_y)
        
        # Inicializar a linha
        line, = ax.plot([], [], 'r-', lw=2, label='Linha de decisão')
        iteration_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
        
        # Função de inicialização para animação
        def init():
            line.set_data([], [])
            iteration_text.set_text('')
            return line, iteration_text
        
        # Função de atualização para animação
        def update(frame):
            w = self.history[frame]
            
            # Atualizar a linha de decisão
            if w[1] != 0:  # Evitar divisão por zero
                x = np.linspace(min_x, max_x, 100)
                y = (0.5 - w[0] * x - w[2]) / w[1]
                line.set_data(x, y)
            
            iteration_text.set_text(f'Iteração: {frame}')
            return line, iteration_text
        
        # Criar animação
        ani = FuncAnimation(fig, update, frames=len(self.history),
                            init_func=init, blit=True, interval=200)
        
        plt.title('Evolução da linha de decisão do Perceptron')
        plt.xlabel('Característica 1')
        plt.ylabel('Característica 2')
        plt.legend()
        plt.grid(True)
        
        return ani
    

In [25]:
def train_and_visualize_and():
    # Dados para a porta lógica AND
    X_and = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y_and = np.array([0, 0, 0, 1])
    
    # Criar e treinar o perceptron
    perceptron = Perceptron(max_iters=100)
    perceptron.fit(X_and, y_and)
    
    # Visualizar o treinamento
    ani = perceptron.visualize_training(X_and, y_and)
    
    # Testar o modelo
    print("Resultados para AND:")
    for x, y_true in zip(X_and, y_and):
        y_pred = perceptron.predict([x])[0]
        print(f"Entrada: {x}, Saída Prevista: {y_pred}, Saída Real: {y_true}")
    
    return perceptron, ani

# Função para treinar o perceptron com a porta lógica OR
def train_and_visualize_or():
    # Dados para a porta lógica OR
    X_or = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y_or = np.array([0, 1, 1, 1])
    
    # Criar e treinar o perceptron
    perceptron = Perceptron(max_iters=100)
    perceptron.fit(X_or, y_or)
    
    # Visualizar o treinamento
    ani = perceptron.visualize_training(X_or, y_or)
    
    # Testar o modelo
    print("Resultados para OR:")
    for x, y_true in zip(X_or, y_or):
        y_pred = perceptron.predict([x])[0]
        print(f"Entrada: {x}, Saída Prevista: {y_pred}, Saída Real: {y_true}")
    
    return perceptron, ani

In [26]:
perceptron_and, ani_and = train_and_visualize_and()
plt.show()

Treine o modelo primeiro usando o método fit()
Resultados para AND:
Entrada: [0 0], Saída Prevista: 0, Saída Real: 0
Entrada: [0 1], Saída Prevista: 0, Saída Real: 0
Entrada: [1 0], Saída Prevista: 0, Saída Real: 0
Entrada: [1 1], Saída Prevista: 1, Saída Real: 1


In [27]:
perceptron_or, ani_or = train_and_visualize_or()
plt.show()

Treine o modelo primeiro usando o método fit()
Resultados para OR:
Entrada: [0 0], Saída Prevista: 0, Saída Real: 0
Entrada: [0 1], Saída Prevista: 1, Saída Real: 1
Entrada: [1 0], Saída Prevista: 1, Saída Real: 1
Entrada: [1 1], Saída Prevista: 1, Saída Real: 1
